In [1]:
import csv
import math

import matplotlib.pyplot as plt
import numpy as np

from keras import applications
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras.initializers import glorot_uniform
from keras.applications.vgg16 import preprocess_input
from keras.models import load_model


import cv2
from matplotlib import pyplot as plt
from skimage import color
from skimage.transform import resize

from IPython.display import Audio
from IPython.display import display
import time

files = ['02', '04', '06', '08', '11', '15', '23', '39', '52', '59', '62', '63', '72']


Using TensorFlow backend.


In [2]:
import csv

with open("pac_transform_guide.csv") as file:
    reader = csv.reader(file)
    i = 0
    image_paths = []
    labels = []
    
    for ind, row in enumerate(reader):
        index = row[0]
        filename = row[1]
        label = row[2]
        image_paths.append(filename)
        labels.append(label)

In [11]:
for file in files:
    print('File: ' + file)
    train_0 = np.load('pac_augment_arrays/ocean_0_train_' + file + '.npy')
    print(len(train_0))
    print(image_paths[min(train_0)])
    print(image_paths[max(train_0)])

    train_1 = np.load('pac_augment_arrays/ocean_1_train_' + file + '.npy')
    print(len(train_1))
    print(image_paths[min(train_1)])
    print(image_paths[max(train_1)])
    
    train_transforms = np.load('pac_augment_arrays/ocean_transforms_train_' + file + '.npy')
    print(len(train_transforms))
    print(image_paths[min(train_transforms)])
    print(image_paths[max(train_transforms)])
    
    test_0 = np.load('pac_augment_arrays/ocean_0_test_' + file + '.npy')
    print(image_paths[min(test_0)])
    print(image_paths[max(test_0)])
    
    test_1 = np.load('pac_augment_arrays/ocean_1_test_' + file + '.npy')
    print(image_paths[min(test_1)])
    print(image_paths[max(test_1)])

File: 02
1829
/pac_augmented/train_02_ocean/0/0.png
/pac_augmented/train_02_ocean/0/1828.png
151
/pac_augmented/train_02_ocean/1/0.png
/pac_augmented/train_02_ocean/1/150.png
1812
/pac_augmented/train_02_ocean/transforms/0_0.png
/pac_augmented/train_02_ocean/transforms/150_11.png
/pac_augmented/test_02_ocean/0/0.png
/pac_augmented/test_02_ocean/0/602.png
/pac_augmented/test_02_ocean/1/0.png
/pac_augmented/test_02_ocean/1/56.png
File: 04
1783
/pac_augmented/train_04_ocean/0/0.png
/pac_augmented/train_04_ocean/0/1782.png
197
/pac_augmented/train_04_ocean/1/0.png
/pac_augmented/train_04_ocean/1/196.png
1773
/pac_augmented/train_04_ocean/transforms/0_0.png
/pac_augmented/train_04_ocean/transforms/196_8.png
/pac_augmented/test_04_ocean/0/0.png
/pac_augmented/test_04_ocean/0/598.png
/pac_augmented/test_04_ocean/1/0.png
/pac_augmented/test_04_ocean/1/60.png
File: 06
1240
/pac_augmented/train_06_ocean/0/0.png
/pac_augmented/train_06_ocean/0/1239.png
740
/pac_augmented/train_06_ocean/1/0.png
/p

In [4]:
print(len(train_0))
print(len(train_1))
print(len(np.concatenate([train_0, train_1])))

1433
547
1980


In [5]:
def image_generator(indices, batch_size):

    num_batches = int(len(indices) / batch_size)
    
    while True:
        for batch_i in range(num_batches):
            if batch_i == num_batches - 1:
                # special case: return as many as possible
                start_i = batch_i * batch_size
                batch_indices = indices[start_i:]
                
                X = np.zeros((len(batch_indices), 224, 224, 3))
                Y = np.zeros((len(batch_indices), 2)) # Change to one-hot
            
            else:
                start_i = batch_i * batch_size
                end_i = start_i + batch_size

                batch_indices = indices[start_i:end_i]

                X = np.zeros((batch_size, 224, 224, 3))
                Y = np.zeros((batch_size, 2)) # Change to one-hot
            
            for i, index in enumerate(batch_indices):
                #img = image.load_img(image_paths[index], target_size=(224, 224))
                '''
                data = np.load(image_paths[int(index)])
                depth_map = data['x'].astype(np.float32)
                orig_img, ocean = depth_map_to_image(depth_map)
                '''
                ocean = image.load_img(image_paths[index], target_size=(224, 224))
                #ocean = resize(ocean, (224, 224), mode='constant')
                
                X[i, :, :, :] = ocean
                # Convert to 1 hot vector
                one_hot = np.zeros(2)
                sanitation_label = labels[int(index)]
                if sanitation_label == '0':
                    one_hot[0] = 1
                elif sanitation_label == '1':
                    one_hot[1] = 1
                Y[i,:] = one_hot
            
            # use vgg16 preprocessing
            X = preprocess_input(X)
            
            yield (X, Y)

In [27]:
def camera_model(camera_num):
    print(camera_num)
    input_tensor = Input(shape=(224,224,3))
    model = applications.VGG16(weights='imagenet', include_top=False, input_tensor = input_tensor)
    
    # build a classifier model to put on top of the convolutional model
    x = model.output
    x = Flatten(input_shape=(model.output_shape[1:]))(x)
    #x = Dropout(0.2)(x)
    x = Dropout(0.05)(x)
    x = Dense(256, activation='relu', kernel_initializer='glorot_uniform')(x)
    x = Dense(2, activation='softmax', name='output', kernel_initializer='glorot_uniform')(x)

    # add new classifier model on top of convolutional base
    new_model = Model(model.input, x)
    
    for layer in new_model.layers[:19]:
        layer.trainable = False
        
    #new_model = load_model('pca_camera_' + camera_num + '.hdf5')
    
    # RMSprop
    #new_model.compile(loss='categorical_crossentropy',
    #              optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])
    new_model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

    minibatch_size = 64#64
    
    train_0 = np.load('pac_augment_arrays/ocean_0_train_' + camera_num + '.npy')
    train_transforms = np.load('pac_augment_arrays/ocean_transforms_train_' + camera_num + '.npy')
    train_indices = np.random.permutation(np.concatenate([train_0, train_transforms]))
    test_0 = np.load('pac_augment_arrays/ocean_0_test_' + camera_num + '.npy')
    test_1 = np.load('pac_augment_arrays/ocean_1_test_' + camera_num + '.npy')
    test_indices = np.random.permutation(np.concatenate([test_0, test_1]))
    
    print(len(train_indices))
    print(len(test_indices))

    

    #train_indices =     
    #test_indices = np.load('pac_arrays/test_' + camera_num + '.npy')

    image_generator(train_indices, minibatch_size)

    epochs = 10
    minibatch_size = 64#64

    train_steps = math.ceil(len(train_indices) / minibatch_size)
    test_steps = math.ceil(len(test_indices) / minibatch_size)
    
    num0 = 0
    num1 = 0
    for i in train_indices:
        if labels[i] == '0':
            num0 += 1
        elif labels[i] == '1':
            num1 += 1
    ratio = int(num0 / num1)
    print(ratio)
    
    num0 = 0
    num1 = 0
    for i in test_indices:
        if labels[i] == '0':
            num0 += 1
        elif labels[i] == '1':
            num1 += 1
    ratio = int(num0 / num1)
    print(ratio)
        
        
        
    
    #class_weight = {0 : 1.,
    #1: ratio}
    class_weight = {0 : 1.,
    1: ratio}

    # fine-tune the model
    history = new_model.fit_generator(
        image_generator(train_indices, minibatch_size),
        steps_per_epoch=train_steps,
        epochs=epochs,
        validation_data=image_generator(test_indices, minibatch_size),
        nb_val_samples=test_steps)

    new_model.save('pca_augment_camera_' + camera_num + '.hdf5')
    
    test_epochs = 1

    history = new_model.fit_generator(
        image_generator(test_indices, minibatch_size),
        steps_per_epoch=test_steps,
        epochs=test_epochs,
        class_weight = class_weight)


In [22]:
camera_model('02')
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))


02
3641
660
1
10
Epoch 1/10


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, epochs=10, steps_per_epoch=57, validation_data=<generator...)`


57/57 [==============================] - 69s - loss: 0.4926 - acc: 0.9593 - val_loss: 0.5594 - val_acc: 0.9599
Epoch 2/10
57/57 [==============================] - 69s - loss: 0.2121 - acc: 0.9842 - val_loss: 0.3876 - val_acc: 0.9738
Epoch 3/10
57/57 [==============================] - 69s - loss: 0.2005 - acc: 0.9846 - val_loss: 0.3789 - val_acc: 0.9751
Epoch 4/10
57/57 [==============================] - 68s - loss: 0.1678 - acc: 0.9867 - val_loss: 0.2973 - val_acc: 0.9793
Epoch 5/10
57/57 [==============================] - 68s - loss: 0.1609 - acc: 0.9878 - val_loss: 0.3705 - val_acc: 0.9751
Epoch 6/10
57/57 [==============================] - 68s - loss: 0.1297 - acc: 0.9905 - val_loss: 0.5178 - val_acc: 0.9641
Epoch 7/10
57/57 [==============================] - 69s - loss: 0.1191 - acc: 0.9916 - val_loss: 0.3112 - val_acc: 0.9793
Epoch 8/10
57/57 [==============================] - 68s - loss: 0.0985 - acc: 0.9934 - val_loss: 0.3039 - val_acc: 0.9793
Epoch 9/10
57/57 [=================

In [24]:
camera_model('04')
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))


04
3556
660
1
9
Epoch 1/10


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, epochs=10, steps_per_epoch=56, validation_data=<generator...)`


56/56 [==============================] - 75s - loss: 0.4203 - acc: 0.9636 - val_loss: 0.5296 - val_acc: 0.9627
Epoch 2/10
56/56 [==============================] - 67s - loss: 0.1586 - acc: 0.9873 - val_loss: 0.3421 - val_acc: 0.9724
Epoch 3/10
56/56 [==============================] - 67s - loss: 0.1373 - acc: 0.9894 - val_loss: 0.3059 - val_acc: 0.9779
Epoch 4/10
56/56 [==============================] - 67s - loss: 0.1289 - acc: 0.9902 - val_loss: 0.3006 - val_acc: 0.9779
Epoch 5/10
56/56 [==============================] - 67s - loss: 0.1235 - acc: 0.9916 - val_loss: 0.2539 - val_acc: 0.9807
Epoch 6/10
56/56 [==============================] - 67s - loss: 0.1151 - acc: 0.9908 - val_loss: 0.3194 - val_acc: 0.9793
Epoch 7/10
56/56 [==============================] - 67s - loss: 0.1092 - acc: 0.9912 - val_loss: 0.2485 - val_acc: 0.9834
Epoch 8/10
56/56 [==============================] - 67s - loss: 0.0630 - acc: 0.9961 - val_loss: 0.2647 - val_acc: 0.9820
Epoch 9/10
56/56 [=================

In [26]:
camera_model('06')
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))


06
1980
660
1
1
Epoch 1/10


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, epochs=10, steps_per_epoch=31, validation_data=<generator...)`


31/31 [==============================] - 43s - loss: 3.0078 - acc: 0.7744 - val_loss: 1.3141 - val_acc: 0.8950
Epoch 2/10
31/31 [==============================] - 43s - loss: 1.6639 - acc: 0.8721 - val_loss: 1.6702 - val_acc: 0.8688
Epoch 3/10
31/31 [==============================] - 43s - loss: 1.2823 - acc: 0.8975 - val_loss: 1.1514 - val_acc: 0.8702
Epoch 4/10
31/31 [==============================] - 43s - loss: 1.0098 - acc: 0.9088 - val_loss: 0.7606 - val_acc: 0.9296
Epoch 5/10
31/31 [==============================] - 43s - loss: 0.5432 - acc: 0.9498 - val_loss: 0.4482 - val_acc: 0.9503
Epoch 6/10
31/31 [==============================] - 43s - loss: 0.4739 - acc: 0.9544 - val_loss: 0.7983 - val_acc: 0.9240
Epoch 7/10
31/31 [==============================] - 43s - loss: 0.3694 - acc: 0.9612 - val_loss: 0.6351 - val_acc: 0.9378
Epoch 8/10
31/31 [==============================] - 43s - loss: 0.3036 - acc: 0.9657 - val_loss: 0.2700 - val_acc: 0.9627
Epoch 9/10
31/31 [=================

In [29]:
camera_model('08')
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))


08
3478
660
1
9
Epoch 1/10


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, epochs=10, steps_per_epoch=55, validation_data=<generator...)`


55/55 [==============================] - 74s - loss: 0.5653 - acc: 0.9573 - val_loss: 0.1279 - val_acc: 0.9890
Epoch 2/10
55/55 [==============================] - 66s - loss: 0.2005 - acc: 0.9849 - val_loss: 0.2565 - val_acc: 0.9820
Epoch 3/10
55/55 [==============================] - 66s - loss: 0.1859 - acc: 0.9879 - val_loss: 0.3130 - val_acc: 0.9793
Epoch 4/10
55/55 [==============================] - 66s - loss: 0.2098 - acc: 0.9851 - val_loss: 0.2674 - val_acc: 0.9807
Epoch 5/10
55/55 [==============================] - 67s - loss: 0.1701 - acc: 0.9886 - val_loss: 0.2738 - val_acc: 0.9807
Epoch 6/10
55/55 [==============================] - 67s - loss: 0.1362 - acc: 0.9915 - val_loss: 0.2282 - val_acc: 0.9807
Epoch 7/10
55/55 [==============================] - 67s - loss: 0.1242 - acc: 0.9919 - val_loss: 0.1918 - val_acc: 0.9834
Epoch 8/10
55/55 [==============================] - 67s - loss: 0.1420 - acc: 0.9902 - val_loss: 0.1781 - val_acc: 0.9890
Epoch 9/10
55/55 [=================

In [30]:
camera_model('11')
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))


11
3726
660
1
25


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, epochs=10, steps_per_epoch=59, validation_data=<generator...)`


Epoch 1/10
59/59 [==============================] - 77s - loss: 0.4008 - acc: 0.9685 - val_loss: 0.5981 - val_acc: 0.9613
Epoch 2/10
59/59 [==============================] - 70s - loss: 0.2066 - acc: 0.9849 - val_loss: 0.2132 - val_acc: 0.9848
Epoch 3/10
59/59 [==============================] - 70s - loss: 0.1449 - acc: 0.9897 - val_loss: 0.2017 - val_acc: 0.9848
Epoch 4/10
59/59 [==============================] - 70s - loss: 0.1334 - acc: 0.9907 - val_loss: 0.2438 - val_acc: 0.9807
Epoch 5/10
59/59 [==============================] - 70s - loss: 0.1385 - acc: 0.9897 - val_loss: 0.3317 - val_acc: 0.9793
Epoch 6/10
59/59 [==============================] - 70s - loss: 0.0929 - acc: 0.9934 - val_loss: 0.2942 - val_acc: 0.9807
Epoch 7/10
59/59 [==============================] - 70s - loss: 0.1068 - acc: 0.9926 - val_loss: 0.2242 - val_acc: 0.9848
Epoch 8/10
59/59 [==============================] - 70s - loss: 0.0991 - acc: 0.9931 - val_loss: 0.2415 - val_acc: 0.9834
Epoch 9/10
59/59 [======

In [31]:
camera_model('15')
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))


15
3952
660
1
219
Epoch 1/10


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, epochs=10, steps_per_epoch=62, validation_data=<generator...)`


62/62 [==============================] - 83s - loss: 0.1150 - acc: 0.9884 - val_loss: 0.0457 - val_acc: 0.9959
Epoch 2/10
62/62 [==============================] - 73s - loss: 0.0657 - acc: 0.9947 - val_loss: 0.0376 - val_acc: 0.9972
Epoch 3/10
62/62 [==============================] - 73s - loss: 0.0122 - acc: 0.9992 - val_loss: 0.0391 - val_acc: 0.9972
Epoch 4/10
62/62 [==============================] - 73s - loss: 0.0122 - acc: 0.9992 - val_loss: 0.0391 - val_acc: 0.9972
Epoch 5/10
62/62 [==============================] - 73s - loss: 0.0122 - acc: 0.9992 - val_loss: 0.0391 - val_acc: 0.9972
Epoch 6/10
62/62 [==============================] - 73s - loss: 0.0162 - acc: 0.9990 - val_loss: 0.0391 - val_acc: 0.9972
Epoch 7/10
62/62 [==============================] - 73s - loss: 0.0122 - acc: 0.9992 - val_loss: 0.0391 - val_acc: 0.9972
Epoch 8/10
62/62 [==============================] - 73s - loss: 0.0122 - acc: 0.9992 - val_loss: 0.0560 - val_acc: 0.9959
Epoch 9/10
62/62 [=================

In [32]:
camera_model('23')
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))


23
2968
660
1
3
Epoch 1/10


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, epochs=10, steps_per_epoch=47, validation_data=<generator...)`


47/47 [==============================] - 66s - loss: 0.9598 - acc: 0.9216 - val_loss: 1.5933 - val_acc: 0.8771
Epoch 2/10
47/47 [==============================] - 58s - loss: 0.6334 - acc: 0.9538 - val_loss: 1.4090 - val_acc: 0.9006
Epoch 3/10
47/47 [==============================] - 58s - loss: 0.4936 - acc: 0.9612 - val_loss: 0.6672 - val_acc: 0.9503
Epoch 4/10
47/47 [==============================] - 58s - loss: 0.3257 - acc: 0.9746 - val_loss: 0.7041 - val_acc: 0.9461
Epoch 5/10
47/47 [==============================] - 58s - loss: 0.3486 - acc: 0.9732 - val_loss: 0.5193 - val_acc: 0.9530
Epoch 6/10
47/47 [==============================] - 58s - loss: 0.3614 - acc: 0.9691 - val_loss: 0.7077 - val_acc: 0.9448
Epoch 7/10
47/47 [==============================] - 58s - loss: 0.2389 - acc: 0.9816 - val_loss: 2.4043 - val_acc: 0.8370
Epoch 8/10
47/47 [==============================] - 58s - loss: 0.2808 - acc: 0.9774 - val_loss: 0.6466 - val_acc: 0.9475
Epoch 9/10
47/47 [=================

In [33]:
camera_model('39')
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))


39
3930
660
1
109
Epoch 1/10


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, epochs=10, steps_per_epoch=62, validation_data=<generator...)`


62/62 [==============================] - 81s - loss: 2.2280 - acc: 0.8579 - val_loss: 0.1336 - val_acc: 0.9917
Epoch 2/10
62/62 [==============================] - 73s - loss: 0.0528 - acc: 0.9967 - val_loss: 0.1558 - val_acc: 0.9903
Epoch 3/10
62/62 [==============================] - 73s - loss: 0.0528 - acc: 0.9967 - val_loss: 0.1336 - val_acc: 0.9917
Epoch 4/10
62/62 [==============================] - 73s - loss: 0.0528 - acc: 0.9967 - val_loss: 0.1558 - val_acc: 0.9903
Epoch 5/10
62/62 [==============================] - 73s - loss: 0.1037 - acc: 0.9927 - val_loss: 0.1558 - val_acc: 0.9903
Epoch 6/10
62/62 [==============================] - 73s - loss: 0.0447 - acc: 0.9972 - val_loss: 0.1558 - val_acc: 0.9903
Epoch 7/10
62/62 [==============================] - 73s - loss: 0.0447 - acc: 0.9972 - val_loss: 0.1336 - val_acc: 0.9917
Epoch 8/10
62/62 [==============================] - 73s - loss: 0.0447 - acc: 0.9972 - val_loss: 0.1336 - val_acc: 0.9917
Epoch 9/10
62/62 [=================

In [34]:
camera_model('52')
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))


52
3116
660
1
6
Epoch 1/10


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, epochs=10, steps_per_epoch=49, validation_data=<generator...)`


49/49 [==============================] - 70s - loss: 1.0324 - acc: 0.9139 - val_loss: 0.8596 - val_acc: 0.9351
Epoch 2/10
49/49 [==============================] - 60s - loss: 0.5923 - acc: 0.9535 - val_loss: 0.8246 - val_acc: 0.9420
Epoch 3/10
49/49 [==============================] - 60s - loss: 0.3897 - acc: 0.9686 - val_loss: 0.7995 - val_acc: 0.9351
Epoch 4/10
49/49 [==============================] - 60s - loss: 0.3703 - acc: 0.9686 - val_loss: 0.5916 - val_acc: 0.9530
Epoch 5/10
49/49 [==============================] - 60s - loss: 0.3303 - acc: 0.9732 - val_loss: 0.8641 - val_acc: 0.9378
Epoch 6/10
49/49 [==============================] - 60s - loss: 0.2607 - acc: 0.9779 - val_loss: 0.7032 - val_acc: 0.9544
Epoch 7/10
49/49 [==============================] - 60s - loss: 0.2199 - acc: 0.9834 - val_loss: 0.5883 - val_acc: 0.9558
Epoch 8/10
49/49 [==============================] - 60s - loss: 0.1961 - acc: 0.9840 - val_loss: 0.8738 - val_acc: 0.9365
Epoch 9/10
49/49 [=================

In [35]:
camera_model('59')
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))


59
3897
660
1
65


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, epochs=10, steps_per_epoch=61, validation_data=<generator...)`


Epoch 1/10
61/61 [==============================] - 74s - loss: 0.2008 - acc: 0.9791 - val_loss: 0.2058 - val_acc: 0.9862
Epoch 2/10
61/61 [==============================] - 73s - loss: 0.0981 - acc: 0.9932 - val_loss: 0.2226 - val_acc: 0.9862
Epoch 3/10
61/61 [==============================] - 73s - loss: 0.0724 - acc: 0.9940 - val_loss: 0.2972 - val_acc: 0.9765
Epoch 4/10
61/61 [==============================] - 73s - loss: 0.0392 - acc: 0.9972 - val_loss: 0.5824 - val_acc: 0.9489
Epoch 5/10
61/61 [==============================] - 73s - loss: 0.0620 - acc: 0.9951 - val_loss: 0.2093 - val_acc: 0.9862
Epoch 6/10
61/61 [==============================] - 73s - loss: 0.0333 - acc: 0.9980 - val_loss: 0.2392 - val_acc: 0.9848
Epoch 7/10
61/61 [==============================] - 73s - loss: 0.0330 - acc: 0.9980 - val_loss: 0.1558 - val_acc: 0.9903
Epoch 8/10
61/61 [==============================] - 73s - loss: 0.0577 - acc: 0.9958 - val_loss: 0.1674 - val_acc: 0.9890
Epoch 9/10
61/61 [======

In [36]:
camera_model('62')
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))


62
2850
660
1
3
Epoch 1/10


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, epochs=10, steps_per_epoch=45, validation_data=<generator...)`


45/45 [==============================] - 65s - loss: 1.3548 - acc: 0.8921 - val_loss: 1.1369 - val_acc: 0.9130
Epoch 2/10
45/45 [==============================] - 56s - loss: 0.6096 - acc: 0.9519 - val_loss: 0.9668 - val_acc: 0.9254
Epoch 3/10
45/45 [==============================] - 56s - loss: 0.4939 - acc: 0.9610 - val_loss: 0.7512 - val_acc: 0.9461
Epoch 4/10
45/45 [==============================] - 57s - loss: 0.3846 - acc: 0.9680 - val_loss: 0.6682 - val_acc: 0.9475
Epoch 5/10
45/45 [==============================] - 57s - loss: 0.3826 - acc: 0.9722 - val_loss: 0.9049 - val_acc: 0.9323
Epoch 6/10
45/45 [==============================] - 57s - loss: 0.4085 - acc: 0.9685 - val_loss: 0.7241 - val_acc: 0.9448
Epoch 7/10
45/45 [==============================] - 57s - loss: 0.2706 - acc: 0.9784 - val_loss: 0.6477 - val_acc: 0.9448
Epoch 8/10
45/45 [==============================] - 57s - loss: 0.2609 - acc: 0.9796 - val_loss: 1.1673 - val_acc: 0.9130
Epoch 9/10
45/45 [=================

In [37]:
camera_model('63')
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))


63
2563
660
1
2
Epoch 1/10


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, epochs=10, steps_per_epoch=41, validation_data=<generator...)`


41/41 [==============================] - 59s - loss: 3.0431 - acc: 0.7939 - val_loss: 1.4268 - val_acc: 0.8743
Epoch 2/10
41/41 [==============================] - 52s - loss: 0.9821 - acc: 0.9216 - val_loss: 1.1973 - val_acc: 0.8964
Epoch 3/10
41/41 [==============================] - 52s - loss: 0.6592 - acc: 0.9456 - val_loss: 0.8035 - val_acc: 0.9378
Epoch 4/10
41/41 [==============================] - 52s - loss: 0.5110 - acc: 0.9593 - val_loss: 1.8300 - val_acc: 0.8439
Epoch 5/10
41/41 [==============================] - 52s - loss: 0.4508 - acc: 0.9639 - val_loss: 0.7323 - val_acc: 0.9406
Epoch 6/10
41/41 [==============================] - 52s - loss: 0.4021 - acc: 0.9688 - val_loss: 0.9255 - val_acc: 0.9254
Epoch 7/10
41/41 [==============================] - 52s - loss: 0.4580 - acc: 0.9680 - val_loss: 0.8610 - val_acc: 0.9420
Epoch 8/10
41/41 [==============================] - 52s - loss: 0.3557 - acc: 0.9731 - val_loss: 0.6777 - val_acc: 0.9489
Epoch 9/10
41/41 [=================

In [39]:
camera_model('72')
#for camera_num in camera_nums:
#    camera_model(camera_num)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))


72
2527
660
1
2
Epoch 1/10


/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:87: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=11, epochs=10, steps_per_epoch=40, validation_data=<generator...)`


40/40 [==============================] - 52s - loss: 2.1104 - acc: 0.8248 - val_loss: 1.1562 - val_acc: 0.8812
Epoch 2/10
40/40 [==============================] - 52s - loss: 1.0250 - acc: 0.9097 - val_loss: 1.1841 - val_acc: 0.9033
Epoch 3/10
40/40 [==============================] - 52s - loss: 0.7592 - acc: 0.9341 - val_loss: 1.0986 - val_acc: 0.8798
Epoch 4/10
40/40 [==============================] - 52s - loss: 0.5143 - acc: 0.9551 - val_loss: 1.1906 - val_acc: 0.8798
Epoch 5/10
40/40 [==============================] - 51s - loss: 0.4331 - acc: 0.9581 - val_loss: 1.5175 - val_acc: 0.8619
Epoch 6/10
40/40 [==============================] - 52s - loss: 0.4604 - acc: 0.9564 - val_loss: 0.9729 - val_acc: 0.9227
Epoch 7/10
40/40 [==============================] - 52s - loss: 0.4608 - acc: 0.9609 - val_loss: 1.1534 - val_acc: 0.8978
Epoch 8/10
40/40 [==============================] - 52s - loss: 0.2839 - acc: 0.9772 - val_loss: 0.8884 - val_acc: 0.9171
Epoch 9/10
40/40 [=================